In [ ]:
"""
This code file is the first step in the entire project.
It firsts extracts the URLs of the 10K and 10Q filings of the companies in the S&P500.
It then extracts the risk factor, the business description and the MDMA section of all the filings
and subsequently cleans these files to create a final Document Matrix.
"""

'\nThis code file is the first step in the entire project.\nIt firsts extracts the URLs of the 10K and 10Q filings of the companies in the S&P500.\nIt then extracts the risk factor, the business description and the MDMA section of all the filings \nand subsequently cleans these files to create a final Document Matrix.\n'

## Extracting the list of URLs

In [ ]:
import requests
from bs4 import BeautifulSoup
import re # import re module for REGEXes
import pandas as pd
import os
from sec_api import QueryApi, RenderApi, ExtractorApi
api_key = '105d85762b9138da11aa136b3313112c93888324437c9aff80c3babfa607ac34'
queryApi = QueryApi(api_key=api_key)
extractorApi = ExtractorApi(api_key)

In [ ]:
# get SP 500 company lists
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500[0]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [ ]:
# create batches of tickers: [[A,B,C], [D,E,F], ...]
# a single batch has a maximum of max_length_of_batch tickers
def create_batches(tickers = [], max_length_of_batch = 100):
    batches = [[]]
    for ticker in tickers:
        if len(batches[len(batches)-1]) == max_length_of_batch:
              batches.append([])
        batches[len(batches)-1].append(ticker)

    return batches

In [ ]:
def download_10K_metadata(tickers = [], start_year = 2021, end_year = 2023):
    batches = create_batches(tickers)
    frames = []
    for year in range(start_year, end_year+1):
        for batch in batches:
            tickers_joined = ', '.join(batch)
            ticker_query = 'ticker:({})'.format(tickers_joined)
            query_string = '{ticker_query} AND filedAt:[{start_year}-01-01 TO {end_year}-12-31] AND formType:"10-K" AND NOT formType:"10-K/A" AND NOT formType:NT'.format(ticker_query=ticker_query, start_year=year, end_year=year)
            query = {
                "query": { "query_string": {
                    "query": query_string,
                    "time_zone": "America/New_York"
                } },
                "from": "0",
                "size": "200",
                "sort": [{ "filedAt": { "order": "desc" } }]
              }

            response = queryApi.get_filings(query)

            filings = response['filings']

            metadata = list(map(lambda f: {'ticker': f['ticker'],
                                            'cik': f['cik'],
                                            'formType': f['formType'],
                                            'year': year,
                                            'filedAt': f['filedAt'],
                                            'filingUrl': f['linkToFilingDetails']}, filings))

            df = pd.DataFrame.from_records(metadata)

            frames.append(df)
    result = pd.concat(frames)
    return result


In [ ]:
dd = download_10K_metadata(sp500[0]['Symbol'])
dd

,ticker,cik,formType,year,filedAt,filingUrl
0,A,1090872,10-K,2021,2021-12-17T17:01:49-05:00,https://www.sec.gov/Archives/edgar/data/109087...
1,AVGO,1730168,10-K,2021,2021-12-17T16:42:51-05:00,https://www.sec.gov/Archives/edgar/data/173016...
2,AMAT,6951,10-K,2021,2021-12-17T16:14:51-05:00,https://www.sec.gov/Archives/edgar/data/6951/0...
3,ADI,6281,10-K,2021,2021-12-03T16:02:52-05:00,https://www.sec.gov/Archives/edgar/data/6281/0...
4,BDX,10795,10-K,2021,2021-11-24T14:57:19-05:00,https://www.sec.gov/Archives/edgar/data/10795/...
...,...,...,...,...,...,...
88,URI,1067701,10-K,2023,2023-01-25T16:21:53-05:00,https://www.sec.gov/Archives/edgar/data/104716...
89,SLB,87347,10-K,2023,2023-01-25T10:10:50-05:00,https://www.sec.gov/Archives/edgar/data/87347/...
0,ZBH,1136869,10-K,2023,2023-02-24T09:46:33-05:00,https://www.sec.gov/Archives/edgar/data/113686...
1,ZION,109380,10-K,2023,2023-02-23T15:53:40-05:00,https://www.sec.gov/Archives/edgar/data/109380...


In [ ]:
def download_10q_metadata(tickers = [], start_year = 2021, end_year = 2023):
    batches = create_batches(tickers)
    frames = []
    for year in range(start_year, end_year+1):
        for batch in batches:
            tickers_joined = ', '.join(batch)
            ticker_query = 'ticker:({})'.format(tickers_joined)
            query_string = '{ticker_query} AND filedAt:[{start_year}-01-01 TO {end_year}-12-31] AND formType:"10-Q" AND NOT formType:"10-Q/A" AND NOT formType:NT'.format(ticker_query=ticker_query, start_year=year, end_year=year)
            query = {
                "query": { "query_string": {
                    "query": query_string,
                    "time_zone": "America/New_York"
                } },
                "from": "0",
                "size": "200",
                "sort": [{ "filedAt": { "order": "desc" } }]
              }

            response = queryApi.get_filings(query)

            filings = response['filings']

            metadata = list(map(lambda f: {'ticker': f['ticker'],
                                            'cik': f['cik'],
                                            'formType': f['formType'],
                                            'year': year,
                                            'filedAt': f['filedAt'],
                                            'filingUrl': f['linkToFilingDetails']}, filings))

            df = pd.DataFrame.from_records(metadata)

            frames.append(df)
    result = pd.concat(frames)
    return result


In [ ]:
ddq = download_10q_metadata(sp500[0]['Symbol'])
ddq

,ticker,cik,formType,year,filedAt,filingUrl
0,AZO,866787,10-Q,2021,2021-12-17T16:41:03-05:00,https://www.sec.gov/Archives/edgar/data/866787...
1,ACN,1467373,10-Q,2021,2021-12-16T06:59:22-05:00,https://www.sec.gov/Archives/edgar/data/146737...
2,CPB,16732,10-Q,2021,2021-12-08T07:30:41-05:00,https://www.sec.gov/Archives/edgar/data/16732/...
3,BBWI,701985,10-Q,2021,2021-12-03T17:05:57-05:00,https://www.sec.gov/Archives/edgar/data/701985...
4,ADSK,769397,10-Q,2021,2021-12-03T16:02:09-05:00,https://www.sec.gov/Archives/edgar/data/769397...
...,...,...,...,...,...,...
1,ZION,109380,10-Q,2023,2023-08-04T12:57:57-04:00,https://www.sec.gov/Archives/edgar/data/109380...
2,ZBH,1136869,10-Q,2023,2023-08-01T16:07:01-04:00,https://www.sec.gov/Archives/edgar/data/113686...
3,ZION,109380,10-Q,2023,2023-05-05T14:47:52-04:00,https://www.sec.gov/Archives/edgar/data/109380...
4,ZTS,1555280,10-Q,2023,2023-05-04T10:51:57-04:00,https://www.sec.gov/Archives/edgar/data/155528...


In [ ]:
url = pd.concat([dd, ddq], axis=0)
url.to_csv("url.csv", index=False)

In [ ]:
url = pd.read_csv('url.csv')

In [ ]:
url = url[['ticker', 'cik', 'formType', 'year', 'filingUrl']]
url.head()

,ticker,cik,formType,year,filingUrl
0,A,1090872,10-K,2021,https://www.sec.gov/Archives/edgar/data/109087...
1,AVGO,1730168,10-K,2021,https://www.sec.gov/Archives/edgar/data/173016...
2,AMAT,6951,10-K,2021,https://www.sec.gov/Archives/edgar/data/6951/0...
3,ADI,6281,10-K,2021,https://www.sec.gov/Archives/edgar/data/6281/0...
4,BDX,10795,10-K,2021,https://www.sec.gov/Archives/edgar/data/10795/...


In [ ]:
urls_10k = url[url['formType'] == '10-K']

In [ ]:
# Creating a dataframe with the firm ticker, the type of form and the sections from the filings
companies = pd.DataFrame(columns = ['ticker', 'cik', 'formType', 'year', 'businessDescription', 'riskFactors', 'mdma'])
count = 0
for i in urls_10k['filingUrl']:
    #get the risk factors section
    item_1A_text = extractorApi.get_section(i, "1A", "text")

    #get the business description
    item_1_text = extractorApi.get_section(i, "1", "text")

    #get the MDMA
    item_7_text = extractorApi.get_section(i, "7", "text")

    row = url.iloc[count]
    companies.loc[count] = [row['ticker'], row['cik'], row['formType'], row['year'], item_1A_text, item_1_text, item_7_text]
    count+=1

In [ ]:
companies['finalText'] = companies['businessDescription'] + companies['riskFactors'] + companies['mdma']

In [ ]:
urls_10q = url[url['formType'] == '10-Q']

In [ ]:
companies2 = pd.DataFrame(columns = ['ticker', 'cik', 'formType', 'year', 'riskFactors', 'mdma'])
count = 0
for i in urls_10q['filingUrl']:
    #get the risk factors section
    item_1A_text = extractorApi.get_section(i, "part2item1a", "text")

    #get the business description
    #item_1_text = extractorApi.get_section(i, "1", "text")

    #get the MDMA
    item_7_text = extractorApi.get_section(i, "part1item2", "text")

    row = url.iloc[count]
    companies2.loc[count] = [row['ticker'], row['cik'], row['formType'], row['year'], item_1A_text, item_7_text]
    count+=1

In [ ]:
companies2['finalText'] = companies2['riskFactors'] + companies2['mdma']

In [ ]:
companies = companies[['ticker', 'cik', 'formType', 'year', 'finalText']]
companies2 = companies2[['ticker', 'cik', 'formType', 'year', 'finalText']]
finaldf = pd.concat([companies, companies2], axis = 0)

In [ ]:
# DATA CLEANING

contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
def expand_contractions(text):
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    return text

#function for removing unicode data :

import unicodedata
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

#function for removing all the scrub words
def scrub_words(text):
    #Replace \xao characters in text
    text = re.sub('\xa0', ' ', text)

    #Replace non ascii / not words and digits
    text = re.sub("(\\W|\\d)",' ',text)

    #Replace new line characters and following text until space
    text = re.sub('\n(\w*?)[\s]', '', text)

    #Remove html markup
    text = re.sub("<.*?>", ' ', text)

    #Remove extra spaces from the text
    text = re.sub("\s+", ' ', text)
    return text

In [ ]:
#using contractions dictionary to make corrections
finaldf['finalText'] = [expand_contractions(re.sub('’', "'", text)) for text in finaldf['finalText']]


In [ ]:
#stripping the words using space
finaldf['finalText'] = [text.strip().lower() for text in finaldf['finalText']]

#removing accented characters
finaldf['finalText'] = [remove_accented_chars(text) for text in finaldf['finalText']]

#re-placing " " " with space
finaldf['finalText'] = [text.replace('"', '') for text in finaldf['finalText']]

# Removing url's from the text
url_reg  = r'[a-z]*[:.]+\S+'
finaldf['finalText'] = [re.sub(url_reg, '', text) for text in finaldf['finalText']]

finaldf['finalText'] = [re.sub(r"\b[a-zA-Z]\b", "", text) for text in finaldf['finalText']]

#removing scrub_words
finaldf['finalText'] = [scrub_words(text) for text in finaldf['finalText']]

#replace spaaces more than one with single space
finaldf['finalText'] = [re.sub("\s+", ' ', text) for text in finaldf['finalText']]

#finding the length of the words in the data
finaldf['count'] = [len(text.split(' ')) for text in finaldf['finalText']]

In [ ]:
stopwords = ['dtd','copyright','llc','en','html','fe','ed','webfilings','e','vk','g','zip code', 'pagebreak','html'
             'w','c','en','table','body','par','value','per','securities','exchange','comission','telephone','number',
             'zip', 'code', 'end', 'page','xbrl','begin','dc','aa','aaa', 'aaa aa','ab','abn','abn amro','abnormal',
             'abo','abs','ac','az','ba','baa','aoci','aol','apb','api','app','ann','anne','amp','amt','anda','bla','bit',
             'bio','bhc','bb','bbb','bbl','bbls','bc','bcf','bcfe','apr','arc','aro','asa','asa','asc','asic','asp','asr',
             'asu','asus','ave','bms','bnp','bny','boe','blvd','bms','boe','bps','bs','btu', 'btus','ca','cad','cal','ccc',
             'cceeff','cdo','cdos','cds','ce','cede','cg','chk','cmsa','col','com','con','conway','ct','dd','de','dan',
             'dana','dea','wti','wto','wv','wyeth','wyoming','xannual','xerox','xi','xii','xiii','xindicate','xiv','xix','xl',
             'xthe','xv','xvi','xvii','xviii', 'xx','xxi','xxx','wi','vt','vs','von''vie','via','vi','var','ta','tab','tam',
             'td','tdr','tdrs','te','sur','ss','sr','sq','sp','sop','sip','sd','sdn','se',
             '__________________________________________ '
            '__________ ']

In [ ]:
def review_to_words(raw_review):

    remove = re.sub(r'\b\w{1,3}\b', '', raw_review) #removing all words less than 4 characters
    letters_only = re.sub("[^a-zA-Z]", " ", remove)
    word = letters_only.lower().split()

    meaningful_words = [w for w in word if not w in stopwords]
    return( " ".join(meaningful_words))


In [ ]:
finaldf = finaldf.reset_index()

In [ ]:
no_of_rows = len(companies)
# Initialize an empty list to hold the clean reviews
stopwords_text = []

# Loop over each review; create an index i that goes from 0 to the length of the movie review list
for i in range(0, no_of_rows):
    stopwords_text.append(review_to_words(finaldf['finalText'][i]))
#   print("Done {}." .format(i))

In [ ]:
finaldf

,index,ticker,cik,formType,year,finalText,count
0,0,A,1090872,10-K,2021,item a risk factors business and strategic ris...,32764
1,1,AVGO,1730168,10-K,2021,item a risk factors our business operations an...,30716
2,2,AMAT,6951,10-K,2021,item a risk factors the following risk factors...,23672
3,3,ADI,6281,10-K,2021,item a risk factors set forth below and elsewh...,23783
4,4,BDX,10795,10-K,2021,item a risk factors an investment in bd involv...,24944
...,...,...,...,...,...,...,...
4455,3018,IBM,51143,10-Q,2022,item a risk factors we amend the following two...,13447
4456,3019,ISRG,1035267,10-Q,2022,item a ri sk factors you should carefully cons...,5524
4457,3020,LVS,1300514,10-Q,2022,item a risk factors we amend the following two...,13502
4458,3021,IQV,1478242,10-Q,2022,item a risk factors in addition to the other i...,8223


In [ ]:
finaldf.to_csv('cleanedText.csv')